In [ ]:
# load data
%pylab
%matplotlib inline
import pandas as pd
from google.colab import files
uploaded = files.upload()


In [ ]:
import io
bonds = pd.read_csv(io.BytesIO(uploaded['selected_bonds.csv']), parse_dates=['Maturity Date', 'Issue Date'])